In [40]:
import matplotlib.pyplot as plt
from keras import Sequential
from keras.layers import Dense
import pandas as pd
from datetime import datetime
from keras.callbacks import TensorBoard
from sklearn.manifold import TSNE
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA


def read_data(data_path):
    data = pd.read_csv(data_path)
    return data

run_csv = read_data("Data/Church_csv/1^_RUN.csv")
run_csv = run_csv.drop(columns=["Unnamed: 0","seconds"])
print(run_csv)

             x        y         z  speed  acceleration     degrees
0      19.6375  119.519  1.055533    0.0           0.0  180.763076
1      19.6375  119.519  1.055533    0.0           0.0  180.766938
2      19.6375  119.519  1.055533    0.0           0.0  179.907911
3      19.6375  119.519  1.055533    0.0           0.0  178.959499
4      19.6375  119.519  1.055533    0.0           0.0  178.292871
...        ...      ...       ...    ...           ...         ...
17645  19.6167  119.761  1.077899    0.0           0.0  180.476128
17646  19.6167  119.761  1.077899    0.0           0.0  171.826245
17647  19.6167  119.761  1.077899    0.0           0.0  146.341278
17648  19.6167  119.761  1.077899    0.0           0.0   93.364190
17649  19.6167  119.761  1.077899    0.0           0.0    0.002950

[17650 rows x 6 columns]


In [41]:
scaler = StandardScaler()
run_csv = scaler.fit_transform(run_csv)
print(run_csv)

[[-1.67520988e+00  7.37636972e-01  1.33261902e-01 -2.48580976e+00
  -1.89462091e-03  2.22502721e-02]
 [-1.67520988e+00  7.37636972e-01  1.33261902e-01 -2.48580976e+00
  -1.89462091e-03  2.22890518e-02]
 [-1.67520988e+00  7.37636972e-01  1.33261902e-01 -2.48580976e+00
  -1.89462091e-03  1.36634841e-02]
 ...
 [-1.67592123e+00  7.45455320e-01  1.45880593e-01 -2.48580976e+00
  -1.89462091e-03 -3.23382009e-01]
 [-1.67592123e+00  7.45455320e-01  1.45880593e-01 -2.48580976e+00
  -1.89462091e-03 -8.55329594e-01]
 [-1.67592123e+00  7.45455320e-01  1.45880593e-01 -2.48580976e+00
  -1.89462091e-03 -1.79277798e+00]]


In [42]:
now = datetime.now()
logdir = "tf_logs/" + now.strftime("%Y%m%d-%H%M%S") + "/"

tensorboard = TensorBoard(log_dir=logdir)

model = Sequential()
encoder = Sequential(name="encoder")
encoder.add(Dense(256, input_dim=run_csv.shape[1], activation='relu'))
encoder.add(Dense(128, activation="relu"))
encoder.add(Dense(64, activation='relu'))

decoder = Sequential()
decoder.add(Dense(64, activation='relu'))
decoder.add(Dense(128, activation="relu"))
decoder.add(Dense(run_csv.shape[1], activation='relu'))

model.add(encoder)
model.add(decoder)

model.compile(loss='mean_squared_error', optimizer='adam', metrics=['accuracy'])
model.fit(run_csv,run_csv,verbose=1,epochs=10, validation_split=0.1,callbacks=[tensorboard])


[[-2.40848981e+00  1.44637224e-03]
 [-2.40850054e+00  1.45815426e-03]
 [-2.40611331e+00 -1.16245937e-03]
 ...
 [-2.31816460e+00 -9.79743305e-02]
 [-2.17094159e+00 -2.59590254e-01]
 [-1.91149126e+00 -5.44405133e-01]]


In [49]:
scaler = StandardScaler()
run_csv = scaler.fit_transform(run_csv)

enc_input =run_csv # model.get_layer(name="encoder").predict(run_csv)


[[-2.40848981e+00  1.44637224e-03]
 [-2.40850054e+00  1.45815426e-03]
 [-2.40611331e+00 -1.16245937e-03]
 ...
 [-2.31816460e+00 -9.79743305e-02]
 [-2.17094159e+00 -2.59590254e-01]
 [-1.91149126e+00 -5.44405133e-01]]


In [50]:
pca=True

if pca:
    scaler = StandardScaler()
    enc_input = scaler.fit_transform(enc_input)
    pca = PCA(n_components=2)
    principalComponents = pca.fit_transform(enc_input)
    print(principalComponents)
    x_val = []
    y_val=[]
    for i in range(principalComponents.shape[0]):
        x_val.append(principalComponents[i][0])
        y_val.append(principalComponents[i][1])
    
    
    plt.scatter(x=x_val,y=y_val, alpha=0.5)
    plt.show()


[[-2.40848981e+00  1.44637224e-03]
 [-2.40850054e+00  1.45815426e-03]
 [-2.40611331e+00 -1.16245937e-03]
 ...
 [-2.31816460e+00 -9.79743305e-02]
 [-2.17094159e+00 -2.59590254e-01]
 [-1.91149126e+00 -5.44405133e-01]]


In [34]:
run2_csv = read_data("Data/Church_csv/2^_RUN.csv")
run2_csv = run2_csv.drop(columns=["Unnamed: 0","seconds"])

scaler = StandardScaler()
run2_csv = scaler.fit_transform(run2_csv)

enc_input = model.predict(run2_csv)


In [35]:
tsne = TSNE(n_components=2, random_state=0)

tsne_obj= tsne.fit_transform(enc_input)
tsne_df = pd.DataFrame({'X':tsne_obj[:,0],
                        'Y':tsne_obj[:,1],
                        })

In [36]:
plt.scatter(x=tsne_df["X"],y=tsne_df["Y"], alpha=0.5)
plt.show()